In [ ]:
import os
import cv2
import numpy as np

In [ ]:
# https://github.com/abhyantrika/nanonets_object_tracking/

from ObjectTracking_DeepSORT.deep_sort import nn_matching
from ObjectTracking_DeepSORT.deep_sort.detection import Detection
from ObjectTracking_DeepSORT.deep_sort.tracker import Tracker
from ObjectTracking_DeepSORT.deep_sort import generate_detections as gdet

In [ ]:
def Object_tracking(output_path):
    # Definition of the parameters
    max_cosine_distance = 0.7
    nn_budget = None
    counter = 0
    
    #initialize deep sort object
    model_filename = './model_data/mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename, batch_size=1)
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    
    objs = './Videos/Objects'
    frames = './Videos/frames'
    uniquePeople = './Videos/Tracked/UniquePeople/'
    
    if not os.path.isdir(uniquePeople):
        os.mkdir(uniquePeople)
    

    objlist = []
    for img in os.listdir(objs):
        if img.endswith('.jpg'):
            objlist.append(img)
        
    objlist = sorted(objlist, key=lambda x: int(x.split('_')[0]))
    
    def findObjs(frameNumber,objlist):
        paths = []
        for obj in objlist:
            if obj.split('_')[0] == frameNumber:
                paths.append(obj)
            elif int(obj.split('_')[0]) > int(frameNumber):
                return paths
        return paths

    framelist = []
    for img in os.listdir(frames):
        if img.endswith('.jpg'):
            framelist.append(img)
    
    framelist = sorted(framelist, key=lambda x: int(x.split('.')[0]))
    
    for img in framelist:
        boxes, scores, names = [], [], []

        frameNumber = img[:img.find('.jpg')]
        imagePath = frames+'/'+img
        original_frame = cv2.imread(imagePath, cv2.COLOR_BGR2RGB)
        original_frame = cv2.cvtColor(original_frame, cv2.COLOR_BGR2RGB)

        paths = findObjs(frameNumber,objlist)

        for path in paths:

            data = path[:path.find('.jpg')].split('_')
            # 0 frames, 1 obj id, 2 centerpoint, 3 min coordinate, 4 max coordinate
            min_coordinate = data[3].split('COMMA')
            xmin = int(min_coordinate[0].replace('(',''))
            ymin = int(min_coordinate[1].replace(')',''))

            max_coordinate = data[4].replace('.jpg','').split('COMMA')
            xmax = int(max_coordinate[0].replace('(',''))
            ymax = int(max_coordinate[1].replace(')',''))


            #(x, y, width, height)
            box = [xmin,ymin,xmax-xmin,ymax-ymin]

            boxes.append(box)
            scores.append(0.9)
            names.append('1')


        # Obtain all the detections for the given frame.
        boxes = np.array(boxes) 
        names = np.array(names)
        scores = np.array(scores)
        features = np.array(encoder(original_frame, boxes))
        detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(boxes, scores, names, features)]

        # Pass detections to the deepsort object and obtain the track information.
        tracker.predict()
        tracker.update(detections)

        # Obtain info from the tracks
        tracked_bboxes = []
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 5:
                continue 
            bbox = track.to_tlbr() 
            class_name = track.get_class() 
            tracking_id = track.track_id 
            index = 1 
            tracked_bboxes.append(bbox.tolist() + [tracking_id, index]) 

        
        # draw detection on frame
        image = original_frame.copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        for b in tracked_bboxes:
            counter += 1
            coor = np.array(b[:4], dtype=np.int32)
            image_h, image_w, _ = image.shape
            fontScale = 0.5
            score = b[4]
            bbox_color = (255,255,255)
            bbox_thick = int(0.6 * (image_h + image_w) / 600)
            c1, c2 = (coor[0], coor[1]), (coor[2], coor[3])
            
            fullPath = uniquePeople+str(score)+'_'+str(counter)+'.jpg'

            if coor[0] < 0:
                coor[0] = 0
            if coor[1] < 0:
                coor[1] = 0
            if coor[2] < 0:
                coor[2] = 0
            if coor[3] < 0:
                coor[3] = 0

            obj = cv2.resize(original_frame[coor[1]:(coor[3]),coor[0]:(coor[2])],(250,500))
            obj = cv2.cvtColor(obj,cv2.COLOR_BGR2RGB)
            cv2.imwrite(fullPath,obj)
        
            cv2.rectangle(image, c1, c2, bbox_color, bbox_thick)
            bbox_mess = '%s: %.2f' % ('person', score)
            t_size = cv2.getTextSize(bbox_mess, 0, fontScale, thickness=bbox_thick//2)[0]
            cv2.rectangle(image, c1, (c1[0] + t_size[0], c1[1] - t_size[1] - 3), bbox_color, -1)  # filled
            cv2.putText(image, bbox_mess, (c1[0], c1[1]-2), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0, 0, 0), bbox_thick//2, lineType=cv2.LINE_AA)
            
        final_out_path = output_path + frameNumber + '.jpg'
        
        if final_out_path != '': 
            cv2.imwrite(final_out_path,image)

    cv2.destroyAllWindows()


In [ ]:
out_path = './Videos/Tracked/'

if not os.path.isdir(out_path):
    os.mkdir(out_path)

Object_tracking(out_path)